In [2]:
import os
import os.path

if(not os.path.exists('/root/.salem_cache')):
    os.mkdir('/root/.salem_cache') # 创建后它隐形的

! cp -r ./dataset/salem_cache /root/.salem_cache/ # 拷贝文件
! ls -ll /root/.salem_cache/  # 检查有无成功

总用量 12
drwxr-xr-x 3 root root 4096 6月  20 06:48 cache
drwxr-xr-x 2 root root 4096 6月  20 06:48 downloads
drwxr-xr-x 5 root root 4096 6月  20 06:53 salem_cache


In [3]:
# 导入模块
from scipy import interpolate
import metpy.calc as mpcalc
from metpy.units import units
#import xesmf as xe
import xarray as xr
import salem
import numpy as np
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import matplotlib as mpl
from wrf import getvar, interplevel, CoordPair, vertcross, ll_to_xy, xy_to_ll
from netCDF4 import Dataset

In [4]:
# WRF数据目录
wrfout_path = './dataset/wrfout/'

In [6]:
# 读取 WRF 模拟数据
wrf_file = Dataset(wrfout_path + 'wrfout_d01_2019-08-09_06_00_00')

In [7]:
# 提取位势高度
z = getvar(wrf_file, 'z')
# 提取高度层
p = getvar(wrf_file, 'pressure')
# 计算500 hPa位势高度
ht_500 = interplevel(z, p, 500)

In [8]:
# 获取海平面气压最低值所在的格点位置
slp = getvar(wrf_file, 'slp')
loc = np.where(np.array(slp == slp.min()) == True)
loc

(array([221]), array([203]))

In [9]:
# 根据格点位置得到该点的经纬度信息
xy_to_ll(wrf_file, 221, 203)

<xarray.DataArray 'latlon' (lat_lon: 2)>
array([ 26.58969234, 122.93884283])
Coordinates:
    xy_coord  object CoordPair(x=221, y=203)
  * lat_lon   (lat_lon) <U3 'lat' 'lon'

In [10]:
qv=getvar(wrf_file,'rh')
qv_500=interplevel(qv,p,500)
qv1=qv_500[221,203]
qv1

<xarray.DataArray 'rh_interp' ()>
array(72.22604, dtype=float32)
Coordinates:
    XLONG    float32 122.4
    XLAT     float32 27.08
    XTIME    float32 720.0
    Time     datetime64[ns] 2019-08-09T06:00:00
    level    int64 500
Attributes:
    FieldType:      104
    units:          %
    stagger:        
    coordinates:    XLONG XLAT XTIME
    projection:     LambertConformal(stand_lon=123.0, moad_cen_lat=27.0, true...
    missing_value:  9.969209968386869e+36
    _FillValue:     9.969209968386869e+36
    vert_units:     hPa